# Transfer learning using InceptionV3 for image classification

## ***Link to download data set: [here](https://github.com/FPT-ThaiTuan/Transfer-Learning-Use-Inception-v3-For-Image-Classification/tree/main)***

## Applied in human and horse data set with accuracy up to 99,39%

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Dataset

## 1. Download data + Process data

In [ ]:
link_train = 'https://storage.googleapis.com/learning-datasets/horse-or-human.zip'
link_validation = 'https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip'

In [ ]:
!pip install wget

In [ ]:
import wget

In [ ]:
wget.download(link_train,out='./Train')
wget.download(link_validation,out='./Validation')

In [ ]:
%cd /content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Dataset/Train

In [ ]:
!unzip -q horse-or-human.zip

In [ ]:
%cd /content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Dataset/Validation

In [ ]:
!unzip -q validation-horse-or-human.zip

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Flatten,Conv2D,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
train_folder_direc = '/content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Dataset/Train'
valid_folder_direc = '/content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Dataset/Validation'

In [ ]:
train_datagen = ImageDataGenerator(
                           rescale=1.0/255.0,     #Normalize values down to [0-1]
                           rotation_range=40,
                           zoom_range=0.2,        #zoom in-out 20%
                           shear_range=0.2,
                           width_shift_range=0.2, #shift 20%
                           height_shift_range=0.2,#shift 20%
                           horizontal_flip=True,  #randomly flip images
                           vertical_flip=True,
                           )

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_datagentor = train_datagen.flow_from_directory(train_folder_direc,
                                                     batch_size=20,
                                                     class_mode='binary',
                                                     target_size=(150,150))


validation_datagentor = validation_datagen.flow_from_directory(valid_folder_direc,
                                                     batch_size=20,
                                                     class_mode='binary',
                                                     target_size=(150,150))

## 2. Transfer Learning using InceptionV3

### Load the weights file of the Inception V3 model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
# Import the inception model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
# Print the mode pre summary
pre_trained_model.summary()

In [ ]:
# Freeze all layers
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
# Print the mode pre summary
pre_trained_model.summary()

In [ ]:
# Select class landmark. At the top it will be fixed, at the back it will be variable
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True


In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:


# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1,activation='sigmoid')(x)

Inception_v3_model = Model( pre_trained_model.input, x)

Inception_v3_model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics =['accuracy'])

In [ ]:
print('Building model...........................')
#model fitting
callbacks =myCallback()
history = Inception_v3_model.fit(train_datagentor,
                              validation_data = validation_datagentor,
                              steps_per_epoch=50,
                              epochs=10,
                              validation_steps=10,
                              verbose=1,
                              callbacks = [callbacks])
print('Success!!!!!!!!!!!!!!!!!!!!!!')

## 3. Show result

In [ ]:
# Plot the loss of training and validation
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


# Plot the accuracy of training and validation
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

## 4. Save model

In [ ]:
%cd /content/drive/MyDrive/AI_project/Transfer_Learning_Using_Inception_V3/Model

In [ ]:
Inception_v3_model.save('InceptionV3.h5')